In [1]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [35]:
from pdf2image import convert_from_path
import pytesseract
import cv2
from pytesseract import Output
import pandas as pd
import numpy as np

In [44]:
def confidenceLevelSegregation(df,confLevel):
    #Lower confidence boxes shown in red
    if int(df['conf']) < confLevel:
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'Low'
    else: # Rest of the boxes are in green
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'High'
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    # df['confidenceLevel'] = np.where(
    #     df['conf'] < confLevel, # condition
    #     "Low",    # result if true
    #     "High",   # result if false
    # )
    # df['confidenceLevel'] = df['conf'].map(lambda x : "Low" if(x < confLevel) else "High")
    return df

In [49]:
def paraBreakdownConfidenceLevel(folderName,pageNum,confLevel):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum

    for i in range(len(grouped_df)):
        if int(grouped_df.loc[i,'left']) < confLevel:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            df['confidenceLevel'] = 'Low'
        else:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, ), 2)
            df['confidenceLevel'] = 'High'
            
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    return df


    # segregatedDF = confidenceLevelSegregation(grouped_df,confLevel)
    # return segregatedDF


In [6]:
def paraBreakdown(folderName,pageNum):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum
    # # Creating a visualization that showcases the boxes 
    # for i in range(len(grouped_df)):
    # # print(grouped_df.loc[i,'left'])
    #     if int(grouped_df.loc[i,'left']) > 60:
    #         (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    #         img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # cv2.imshow('img', img)
    # cv2.waitKey(5000)
    # grouped_df.to_csv(f'./Page_{pageNum}paraBlocksTrial.csv')

    # display(grouped_df)
    return grouped_df

In [48]:
#Converting pdf pages into individual images
def convertPDFtoImages(pdfPath,pdfFolderToStoreImages,outputFilePath,confLevel):
    pdfs = pdfPath #"./Articles/amz_letters.pdf"
    pages = convert_from_path(pdfs, 350)
    print('Ingested all the pages from the pdf. Moving on to process them next ==>')
    df = pd.DataFrame()
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(f'{pdfFolderToStoreImages}/{image_name}', "JPEG")
        print(f'Doing iteration number {i}')
        res = paraBreakdownConfidenceLevel(pdfFolderToStoreImages,i,confLevel)
        # display(res)
        df=pd.concat([df,res], ignore_index=True)
        i = i+1
    display(df)
    df.to_csv(f'{outputFilePath}')

In [50]:
convertPDFtoImages('./Articles/amz_letters.pdf','AMZ_Letters','AMZ_Letters/fullAMZParaBlocks.csv',30)

NameError: name 'convertPDFtoImages' is not defined